# Unión Inteligente de Tablas Excel

Este programa realiza una **unión progresiva e inteligente** de múltiples archivos Excel `(.xlsx)` ubicados en la carpeta `./input`, comparando sus columnas para identificar similitudes y diferencias entre ellas. El objetivo es automatizar la **identificación de campos comunes y únicos** entre tablas y generar reportes comprensibles en Excel que describan estas comparaciones.

Funcionalidad Principal

1. Lectura de Archivos:
    Todos los archivos .xlsx del directorio ./input son leídos y procesados en el orden en que se encuentran.

2. Comparación Progresiva:
    Se toma la primera tabla como base. Luego, se compara esta tabla con la segunda para detectar:

    * Columnas únicas de la primera tabla.

    * Columnas compartidas entre ambas tablas.

    * Columnas únicas de la segunda tabla.

3. Creación de Reporte Comparativo:
    Por cada comparación se genera un archivo Excel con tres columnas:

| Únicas en Tabla A | Compartidas | Únicas en Tabla B |

4. Unión Acumulativa:
Posterior a cada comparación, se crea una nueva tabla unificada que contiene las columnas de ambas tablas. Esta nueva tabla se usa como base para compararla con la siguiente tabla del directorio, repitiendo el proceso hasta que se hayan procesado todas.

5. Salida:
Los reportes generados se guardan en la carpeta `./output`, con nombres como `comparacion_1_2.xlsx`, `comparacion_1_2_3.xlsx`, etc., indicando el número de tablas que han sido integradas hasta ese punto.

In [2]:
import os
import pandas as pd

# Crear carpetas de entrada y salida si no existen
os.makedirs('./input', exist_ok=True)
os.makedirs('./output', exist_ok=True)

# Listar archivos .xlsx en la carpeta ./input
excel_files = [f for f in os.listdir('./input') if f.endswith('.xlsx')]

# Mostrar los archivos encontrados
print("Archivos encontrados en ./input:")
for idx, file in enumerate(excel_files, 1):
    print(f"{idx}: {file}")

Archivos encontrados en ./input:
1: RS_LINFOL.xlsx
2: FC_FOLIOS.xlsx
3: RS_PROLIN.xlsx


In [3]:
# Leer todas las tablas Excel en dataframes
dataframes = [pd.read_excel(f'./input/{fname}') for fname in excel_files]

# Inicializar la tabla base con la primera
tabla_base = dataframes[0]
columnas_base = set(tabla_base.columns)

# Bandera
bandera = False

for i in range(1, len(dataframes)):
    tabla_nueva = dataframes[i]
    columnas_nueva = set(tabla_nueva.columns)

    # Obtener nombres de las tablas para el reporte
    nombre_a = excel_files[i-1].replace('.xlsx', '')
    nombre_b = excel_files[i].replace('.xlsx', '')
    if bandera:
        nombre_a = nom_reporte

    # Comparar columnas
    unicas_base = sorted(columnas_base - columnas_nueva)
    compartidas = sorted(columnas_base & columnas_nueva)
    unicas_nueva = sorted(columnas_nueva - columnas_base)

    # Crear reporte comparativo
    max_len = max(len(unicas_base), len(compartidas), len(unicas_nueva))
    reporte = pd.DataFrame({
        f'Únicas en {nombre_a}': unicas_base + [''] * (max_len - len(unicas_base)),
        'Compartidas': compartidas + [''] * (max_len - len(compartidas)),
        f'Únicas en {nombre_b}': unicas_nueva + [''] * (max_len - len(unicas_nueva)),
    })

    # Guardar reporte
    nom_reporte = f"{nombre_a}_{nombre_b}"
    bandera = True
    nombre_reporte = f'./output/comparacion_{nombre_a}_{nombre_b}.xlsx'
    reporte.to_excel(nombre_reporte, index=False)

    # Convertir los tipos de columnas compartidas a 'object' para evitar errores de tipo en el merge
    for col in compartidas:
        tabla_base[col] = tabla_base[col].astype('object')
        tabla_nueva[col] = tabla_nueva[col].astype('object')

    # Unir tablas para la siguiente iteración
    tabla_base = pd.merge(tabla_base, tabla_nueva, how='outer')
    columnas_base = set(tabla_base.columns)

/home/server-test/Documentos/Practicante-Practicando/Aprendiendo-Python/unionTablas/.venv/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/server-test/Documentos/Practicante-Practicando/Aprendiendo-Python/unionTablas/.venv/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/server-test/Documentos/Practicante-Practicando/Aprendiendo-Python/unionTablas/.venv/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
# Crear carpeta de salida para selección si no existe
os.makedirs('./output-seleccion', exist_ok=True)

# Mostrar menú para seleccionar dos tablas
print("Seleccione dos tablas para comparar y unir:")
for idx, file in enumerate(excel_files, 1):
    print(f"{idx}: {file}")

try:
    idx1 = int(input("Ingrese el número de la primera tabla: ")) - 1
    idx2 = int(input("Ingrese el número de la segunda tabla: ")) - 1
    if idx1 == idx2 or idx1 not in range(len(excel_files)) or idx2 not in range(len(excel_files)):
        raise ValueError("Selección inválida.")
except Exception as e:
    print(f"Error en la selección: {e}")
else:
    tabla_a = pd.read_excel(f'./input/{excel_files[idx1]}')
    tabla_b = pd.read_excel(f'./input/{excel_files[idx2]}')
    columnas_a = set(tabla_a.columns)
    columnas_b = set(tabla_b.columns)

    unicas_a = sorted(columnas_a - columnas_b)
    compartidas = sorted(columnas_a & columnas_b)
    unicas_b = sorted(columnas_b - columnas_a)

    max_len = max(len(unicas_a), len(compartidas), len(unicas_b))
    nombre_a = excel_files[idx1].replace('.xlsx', '')
    nombre_b = excel_files[idx2].replace('.xlsx', '')

    reporte = pd.DataFrame({
        f'Únicas en {nombre_a}': unicas_a + [''] * (max_len - len(unicas_a)),
        'Compartidas': compartidas + [''] * (max_len - len(compartidas)),
        f'Únicas en {nombre_b}': unicas_b + [''] * (max_len - len(unicas_b)),
    })

    # Guardar reporte
    nombre_reporte = f'./output-seleccion/comparacion_{nombre_a}_{nombre_b}.xlsx'
    reporte.to_excel(nombre_reporte, index=False)
    print(f"Reporte guardado en: {nombre_reporte}")

    # Convertir tipos de columnas compartidas a 'object'
    for col in compartidas:
        tabla_a[col] = tabla_a[col].astype('object')
        tabla_b[col] = tabla_b[col].astype('object')

    # Unir tablas y mostrar preview
    tabla_unida = pd.merge(tabla_a, tabla_b, how='outer')
    print("Vista previa de la tabla unida:")
    display(tabla_unida.head())

Seleccione dos tablas para comparar y unir:
1: RS_LINFOL.xlsx
2: FC_FOLIOS.xlsx
3: RS_PROLIN.xlsx


/home/server-test/Documentos/Practicante-Practicando/Aprendiendo-Python/unionTablas/.venv/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/server-test/Documentos/Practicante-Practicando/Aprendiendo-Python/unionTablas/.venv/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Reporte guardado en: ./output-seleccion/comparacion_RS_PROLIN_RS_LINFOL.xlsx
Vista previa de la tabla unida:


,XHOTEL_ID,XRESERVA_ID,XNUM_LINEA,XNUM_LINEA_ID,XFECHA_SERV,XCONCEPTO_ID,XCANTIDAD,XIMPORTE,XPRECIO_SIN,XMONEDA,...,FOREIGN_FACTOR,LOCAL_FACTOR,XSUP_MODIF,XZONH_CARGO,ZONH_DTO,XES_PULSERA,XPULSERA_OK,ZZMISC_ID,ZZRES_CARGO_ORIG,ZZTMS_INV_CRNOTE
0,C101,15250,15,1,2024-06-07,MI_TIPS,1,40000,40000,USD,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C101,15831,39,1,2025-01-01,MI_TIPS,1,40000,40000,USD,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C101,15833,39,1,2025-01-01,MI_TIPS,1,40000,40000,USD,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C101,29511,15,1,2024-11-07,MI_TIPS,1,40000,40000,USD,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C101,33293,23,1,2025-01-01,MI_TIPS,1,40000,40000,USD,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
